<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# [Reconnaissance d'Activité Humaine](https://github.com/wikistat/Ateliers-Big-Data/5-HumanActivityRecognition) ([*HAR*](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones)) en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a>   Seconde partie:  étude des signaux bruts -  Exploration et prédiction avec les coefficients dans une base d'ondelettes.



#   Introduction
##  Contexte
Les données sont issues de la communauté qui vise la reconnaissance d'activités humaines (*Human activity recognition, HAR*) à partir d’enregistrements, par exemple du gyroscope et de l'accéléromètre d'un smartphone, objet connecté précurseur et dont la fonctionnalité de téléphonie devient très secondaire.
Voir à ce propos l'[article](https://www.elen.ucl.ac.be/Proceedings/esann/esannpdf/es2013-11.pdf) relatant un colloque de 2013.  

Les données publiques disponibles et largement étudiées ont été acquises, décrites et analysées par [Anguita et al. (2013)]().
Elles sont accessibles sur le [dépôt](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) de l'University California Irvine (UCI) consacré à l'apprentissage machine ainsi que sur le site *Kaggle*.

L'archive contient les données brutes: accélérations en x, y, et z, chacun de 128 colonnes. D'autres fichiers en y soustrayant la gravité naturelle ainsi que les accélérations angulaires en x, y, et z soit en tout 9 fichiers. Mais 6 utiles avec 6*128=768 mesures.

C'est sur ces données brutes que nous allons appliquer des méthodes d'apprentissage.

## Objectifs
Cette deuxième étape s'intéresse aux données brutes. Est-il possible d'économiser le travail préliminaire de définition des variables métier en utilisant par exemple les ressources de décompositions systématiques sur une base d'ondelette ou un algorihtme d'apprentissage profond? L'enjeu est important, le calcul en temps réel des variables métier consomme beaucoup d'énergie et l'objectif serait de pouvoir embarquer (faire porter) un système économe défini par exemple par un réseau de neurones après son apprentissage.

Comme pour les données métier, l'étude commence par une exploration: affichage et visualisation des données, réduction de dimension.

## Librairies

In [ ]:
import pandas as pd
import numpy as np
import random
import itertools
import collections
import time
import utils.load as ul


# Plot et Display
import utils.illustration as uil
from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()
sb.set_style("whitegrid")

# Exploration
import sklearn.decomposition as sdec 
import sklearn.preprocessing as sprep
import sklearn.discriminant_analysis as sda

#Prediction
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb




# Prise en charge des données
## Source

Les données sont celles originales du dépôt de l'[UCI](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones). Elle peuvent être téléchargées en cliquant [ici](https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip).

Elles contiennent deux jeux de dimensions différentes, chacun partagé en apprentissage et test.

1. Multidimensionel: un individus est constitué de 9 Séries Temporelles de *dimensions* $(N, 128, 9)$
2. Unidimensionnel: Les 9 Séries Temporelles sont concaténées pour constituer un vecteur de 128*9 = 1152 variables de *dimensions* $(N, 1152)*
        
Deux objets différents sont construits pour définir la variable $Y$ réponse car les librairies `Scikit-learn` et `Keras` prennent en compte des structures différentes: 
    
1. `Scikit-Learn`  Un vecteur de dimension $(N, 1)$ avec, pour chaque individu le numéro du label de l'activité de 0 à 5.
2. `Keras` Une matrice de dimension $(N, 6)$ des indicatrices (0 ou 1) des modalités de $Y$.

## Structurer les données
Définir le chemin d'accès aux données puis les fonctions utiles.

In [ ]:
SIGNALS = [ "body_acc_x", "body_acc_y", "body_acc_z", "body_gyro_x", "body_gyro_y", "body_gyro_z", "total_acc_x", "total_acc_y", "total_acc_z"]
CMAP = plt.get_cmap("Set1")
ACTIVITY_DIC = {1 : "WALKING",
2 : "WALKING UPSTAIRS",
3 : "WALKING DOWNSTAIRS",
4 : "SITTING",
5 : "STANDING",
6 : "LAYING"}
LABELS = [ACTIVITY_DIC[c] for c in range(1,7)]
COLOR_DIC = {v:CMAP(k-2) if v!="WALKING" else CMAP(10) for k,v in ACTIVITY_DIC.items()}


In [ ]:
#Multidimensional Data
X_train, X_test = ul.load_signals("train", SIGNALS), ul.load_signals("test", SIGNALS)
N_train, N_dim, N_signaux  = X_train.shape
N_test, _, _  = X_test.shape


#Flatten_data
X_train_flatten = X_train.reshape((N_train, N_dim*N_signaux), order="F")
X_test_flatten = X_test.reshape((N_test, N_dim*N_signaux), order="F")

# Label Vector
Y_train, Y_test = ul.load_y("train"), ul.load_y("test")
N_per_activity_train = collections.Counter(Y_train)



Y_train_label = np.array([ACTIVITY_DIC[y] for y in Y_train])
Y_test_label = np.array([ACTIVITY_DIC[y] for y in Y_test])

Vérification des dimensions

In [ ]:
print("Dimension")
print("Données Multidimensionelles, : " + str(X_train.shape))
print("Données Unimensionelles, : " + str(X_train_flatten.shape))
print("Vecteur réponse : " + str(Y_train_label.shape))


# Visualisations

## Tous les signaux

In [ ]:
nb_sample_per_activity = dict([(v,50) for k,v in ACTIVITY_DIC.items()])
linestyle_per_activity = dict([(v,"solid") for k,v in ACTIVITY_DIC.items()])
linewidth_per_activity = dict([(v,1)for k,v in ACTIVITY_DIC.items()])

fig = plt.figure(figsize=(18,18))    
uil.plot_signaux(fig, X_train, Y_train_label, SIGNALS, COLOR_DIC, nb_sample_per_activity, 
             linestyle_per_activity, linewidth_per_activity, figdim1 = 3, figdim2 = 3, shuffle=True, legend=False)


## Par signal 

In [ ]:
fig = plt.figure(figsize=(15,8))
nb_sample = 10
isignal=0    
uil.plot_signal_per_activity(fig, X_train, Y_train_label, nb_sample, isignal, SIGNALS, COLOR_DIC)

**Q** Observer l'un des signaux, par exemple `Walking upstairs`. Qu'est ce qui fait que, pour ces signaux ou courbes, une métrique euclidienne classique ($L_2$) est inopérante? 

**Q** Corrélativement pourquoi est-il intéressant de décomposer un signal dans le domaine des fréquances?

# Analyse en composantes principales
Les ACP ne sont pas réduites. Refaire les calculs après réduction des variables : signaux.

## Sur un Signal

In [ ]:
pca = sdec.PCA()
isignal = 4
signal = SIGNALS[isignal]
print("ACP Sur signal : " +signal)
X_r = pca.fit_transform(X_train[:,:,isignal])

In [ ]:
fig = plt.figure(figsize=(15,10))
uil.plot_variance_acp(fig, pca, X_r, whis=100)
fig.suptitle("Résultat ACP sur Signal : " + signal, fontsize=25)

**Attention**: les diagrammes boîtes sont très perturbés par les distributions des composantes avec une très forte concentration autour de 0 et énormément de valeurs atypiques. D'où l'utilisation du paramètre `whis=100` pour rallonger les moustaches.

In [ ]:
colors=[COLOR_DIC[y] for y in Y_train_label]
markersizes = [20 for _ in range(N_train)]
fig = plt.figure(figsize=(10,10), )
ax = fig.add_subplot(1,1,1)
uil.plot_pca(ax,X_r, pca, 1, 2, colors, markersizes)

## Sur tous les signaux

In [ ]:
pca = sdec.PCA()
print("ACP Sur tous les signaux")
X_r = pca.fit_transform(X_train_flatten)

In [ ]:
fig = plt.figure(figsize=(15,10))
uil.plot_variance_acp(fig, pca, X_r)
fig.suptitle("Résultat ACP sur tous les signaux", fontsize=25)

In [ ]:
fig = plt.figure(figsize=(10,10), )
ax = fig.add_subplot(1,1,1)
uil.plot_pca(ax,X_r, pca, 1, 2, colors, markersizes)

# 5 Analyse factorielle discriminante

## Sur un signal

In [ ]:
isignal = 0
signal = SIGNALS[isignal]
print("LDA Sur signal : " +signal)

method = sda.LinearDiscriminantAnalysis() 
lda=method.fit(X_train[:,:,isignal],Y_train_label)
X_r2=lda.transform(X_train[:,:,isignal])

In [ ]:
fig = plt.figure(figsize= (20,20))
count = 0
for nbc, nbc2, count in [(1,2,1), (2,3,2), (3,4,3), (1,3,4), (2,4,5), (1,4,7)] :
    ax = fig.add_subplot(3,3,count)
    uil.plot_pca(ax,X_r2, lda, nbc, nbc2, colors, markersizes)
plt.show()

##  Sur tous les signaux

In [ ]:
print("LDA Sur tous les signaux")

method = sda.LinearDiscriminantAnalysis() 
lda=method.fit(X_train_flatten,Y_train_label)
X_r2=lda.transform(X_train_flatten)

fig = plt.figure(figsize= (20,20))
count = 0
for nbc, nbc2, count in [(1,2,1), (2,3,2), (3,4,3), (1,3,4), (2,4,5), (1,4,7)] :
    ax = fig.add_subplot(3,3,count)
    uil.plot_pca(ax,X_r2, lda, nbc, nbc2, colors, markersizes)
plt.show()

# Prediction sur les signaux

On effectue des prédictions directement sur la concatenation de tous les signaux avec les méthodes de SVM (lineaire et noyaux gaussien) ainsi que Random Forest et Gradient Boosting.

Contrairement aux données métiers on observe que l'on obtient des meilleurs résultats avec des méthodes non-linéaires.

## Svm Linéaire

In [ ]:
ts = time.time()
method = LinearSVC()
method.fit(X_train_flatten,Y_train)
score = method.score(X_test_flatten, Y_test)
ypred = method.predict(X_test_flatten)
ypred_label = np.array([ACTIVITY_DIC[y] for y in ypred])
te = time.time()
t_total = te-ts

print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.crosstab(Y_test_label, ypred_label, rownames=['True'], colnames=['Pred'])

## SVM Noyau Gaussien

In [ ]:
ts = time.time()
method = SVC()
method.fit(X_train_flatten, Y_train)
score = method.score(X_test_flatten, Y_test)
ypred = method.predict(X_test_flatten)
ypred_label = np.array([ACTIVITY_DIC[y] for y in ypred])
te = time.time()
t_total = te-ts

print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.crosstab(Y_test_label, ypred_label, rownames=['True'], colnames=['Pred'])

## Random Forest

In [ ]:
ts = time.time()
method = RandomForestClassifier(n_estimators = 300, n_jobs=-1)
method.fit(X_train_flatten, Y_train)
score = method.score(X_test_flatten, Y_test)
ypred = method.predict(X_test_flatten)
ypred_label = np.array([ACTIVITY_DIC[y] for y in ypred])
te = time.time()
t_total = te-ts

print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.crosstab(Y_test_label, ypred_label, rownames=['True'], colnames=['Pred'])

## Xgboost

In [ ]:
ts = time.time()
method = xgb.XGBClassifier()
method.fit(X_train_flatten, Y_train)
score = method.score(X_test_flatten, Y_test)
ypred = method.predict(X_test_flatten)
ypred_label = np.array([ACTIVITY_DIC[y] for y in ypred])
te = time.time()
t_total = te-ts

print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.crosstab(Y_test_label, ypred_label, rownames=['True'], colnames=['Pred'])

**Q.** Comparer ces performances avec celles obtenues dans la première partie sur les données métier.


L'objectif de la section suivante est de voir si, à partir d'une décomposition des signaux sur une base d'ondelettes, on peut améliorer la qualité de prédiction. 

# Décomposition en ondelettes

In [ ]:
import pywt
from pywt import wavedec


## Illustration des coefficients

### Coefficients pour un signal par comportement

In [ ]:
sample_to_plot = 1
index_per_act_dict = dict([(act, np.where(Y_train==act)[0][:sample_to_plot]) for act in range(1,7)])

In [ ]:
fig = plt.figure(figsize=(15,8))
for ip, (act , index) in enumerate(index_per_act_dict.items()):
    ax=fig.add_subplot(2,3,ip+1)
    coef = pywt.wavedec(X_train_flatten[index,:1024], 'db1')
    uil.coef_pyramid_plot(ax, coef[1:]) ;

    ax.set_title(ACTIVITY_DIC[act]);
fig.tight_layout()
plt.show()

**Q.** Quelle base d'ondelettes est utilisée ?  Remarquez vous des différences de comportement des coefficients d ondelettes selon les 
classes ?

### Boxplot des coefficients par niveau

In [ ]:
X_train_db_list = pywt.wavedec(X_train_flatten, 'db1')
uil.plot_boxplot_coef_concat_per_signal(X_train_db_list, Y_train, labels=LABELS, activity_dic=ACTIVITY_DIC, 
                             color_dic=COLOR_DIC, N_per_activity_train= N_per_activity_train)

**Q.** Ces boxplots sont-ils cohérents avec les observations précédentes sur un signal par classe ? 

## ACP sur les coefficients d'ondelettes

### Avec tous les coefficients

In [ ]:
X_train_db = np.concatenate(pywt.wavedec(X_train_flatten, 'db1'), axis=1)
X_test_db = np.concatenate(pywt.wavedec(X_test_flatten, 'db1'), axis=1)

## ACP 
pca = sdec.PCA()
X_train_db_pca = pca.fit_transform(X_train_db)

In [ ]:
fig = plt.figure(figsize=(15,10))
uil.plot_variance_acp(fig, pca, X_train_db_pca)

In [ ]:
fig = plt.figure(figsize= (20,20))
count = 0
for nbc, nbc2, count in [(1,2,1), (2,3,2), (3,4,3), (1,3,4), (2,4,5), (1,4,7)] :
    ax = fig.add_subplot(3,3,count)
    uil.plot_pca(ax,X_train_db_pca, pca, nbc, nbc2, colors, markersizes)
plt.show()

**Q.** Comparer ces résultats avec l'ACP sur tous les signaux réalisée précédemment.

### Avec les coefficients seuillés

In [ ]:
cA10, cD10, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = pywt.wavedec(X_train_flatten, 'db1')

sigma=0.01
thresh = sigma*np.sqrt(2*np.log((X_train_flatten.shape[1])))

# On seuille seulement les coefficients de détail : 
cD10=pywt.threshold(cD10, thresh, 'soft')
cD9=pywt.threshold(cD9, thresh, 'soft')
cD8=pywt.threshold(cD8, thresh, 'soft')
cD7=pywt.threshold(cD7, thresh, 'soft')
cD6=pywt.threshold(cD6, thresh, 'soft')
cD5=pywt.threshold(cD5, thresh, 'soft')
cD4=pywt.threshold(cD4, thresh, 'soft')
cD3=pywt.threshold(cD3, thresh, 'soft')
cD2=pywt.threshold(cD2, thresh, 'soft')
cD1=pywt.threshold(cD1, thresh, 'soft')

X_train_dbth = np.concatenate((cA10, cD10, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1), axis=1)

In [ ]:
cA10, cD10, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = pywt.wavedec(X_test_flatten, 'db1')

sigma=0.01
thresh = sigma*np.sqrt(2*np.log((X_test_flatten.shape[1])))

# On seuille seulement les coefficients de détail : 
cD10=pywt.threshold(cD10, thresh, 'soft')
cD9=pywt.threshold(cD9, thresh, 'soft')
cD8=pywt.threshold(cD8, thresh, 'soft')
cD7=pywt.threshold(cD7, thresh, 'soft')
cD6=pywt.threshold(cD6, thresh, 'soft')
cD5=pywt.threshold(cD5, thresh, 'soft')
cD4=pywt.threshold(cD4, thresh, 'soft')
cD3=pywt.threshold(cD3, thresh, 'soft')
cD2=pywt.threshold(cD2, thresh, 'soft')
cD1=pywt.threshold(cD1, thresh, 'soft')

X_test_dbth = np.concatenate((cA10, cD10, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1), axis=1)

In [ ]:
## ACP 
pca = sdec.PCA()
X_train_dbth_pca = pca.fit_transform(X_train_dbth)

In [ ]:
fig = plt.figure(figsize=(15,10))
uil.plot_variance_acp(fig, pca, X_train_db_pca)

In [ ]:
fig = plt.figure(figsize= (20,20))
count = 0
for nbc, nbc2, count in [(1,2,1), (2,3,2), (3,4,3), (1,3,4), (2,4,5), (1,4,7)] :
    ax = fig.add_subplot(3,3,count)
    uil.plot_pca(ax,X_train_dbth_pca, pca, nbc, nbc2, colors, markersizes)
plt.show()

**Q.** Comparez avec les résultats précédents ? Les différences sont-elles significatives ? Essayez de modifier le seuil, puis de ne pas seuiller les premiers niveaux de détail. 

# Prédiction des classes avec les coefficients d'ondelettes

## SVM Linéaire

En considérant tous les coefficients, le score est identique (56.73% contre 56.93%) aux données brutes.
On obtient un net gain en utilisant les coefficients seuillés : 66.34%

Cependant ces scores sont toujours loins des 96% des données métiers

In [ ]:
ts = time.time()
method = LinearSVC()
method.fit(X_train_db, Y_train)
score = method.score(X_test_db, Y_test)
ypred = method.predict(X_test_db)
te = time.time()
t_total = te-ts

print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.DataFrame(confusion_matrix(Y_test, ypred), index = LABELS, columns=LABELS)

In [ ]:
ts = time.time()
method = LinearSVC()
method.fit(X_train_dbth, Y_train)
score = method.score(X_test_dbth, Y_test)
ypred = method.predict(X_test_dbth)
te = time.time()
t_total = te-ts

print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.DataFrame(confusion_matrix(Y_test, ypred), index = LABELS, columns=LABELS)

## SVM Noyau Gaussien

En utilisant un noyau non lineaire, les résultats sont globalement meilleurs. (> 70%)
Cependant l'utilisation de cette méthode sur les coefficients d'ondelettes (seuillés ou non) produit de moins bons scores (resp 72.07 et 74.34) que sur les signaux bruts (76.96)

In [ ]:
ts = time.time()
method = SVC()
method.fit(X_train_db, Y_train)
score = method.score(X_test_db, Y_test)
ypred = method.predict(X_test_db)
te = time.time()
t_total = te-ts

print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.DataFrame(confusion_matrix(Y_test, ypred), index = LABELS, columns=LABELS)

In [ ]:
ts = time.time()
method = SVC()
method.fit(X_train_dbth, Y_train)
score = method.score(X_test_dbth, Y_test)
ypred = method.predict(X_test_dbth)
te = time.time()
t_total = te-ts

print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.DataFrame(confusion_matrix(Y_test, ypred), index = LABELS, columns=LABELS)

## Random Forest

In [ ]:
ts = time.time()
method = RandomForestClassifier()
method.fit(X_train_db, Y_train)
score = method.score(X_test_db, Y_test)
ypred = method.predict(X_test_db)
te = time.time()
t_total = te-ts

print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.DataFrame(confusion_matrix(Y_test, ypred), index = LABELS, columns=LABELS)

In [ ]:
ts = time.time()
method = RandomForestClassifier()
method.fit(X_train_dbth, Y_train)
score = method.score(X_test_dbth, Y_test)
ypred = method.predict(X_test_db)
te = time.time()
t_total = te-ts

print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.DataFrame(confusion_matrix(Y_test, ypred), index = LABELS, columns=LABELS)

## Xgboost

In [ ]:
ts = time.time()
method = xgb.XGBClassifier()
method.fit(X_train_db, Y_train)
score = method.score(X_test_db, Y_test)
ypred = method.predict(X_test_db)
ypred_label = np.array([ACTIVITY_DIC[y] for y in ypred])
te = time.time()
t_total = te-ts

print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.crosstab(Y_test_label, ypred_label, rownames=['True'], colnames=['Pred'])

In [ ]:
ts = time.time()
method = xgb.XGBClassifier()
method.fit(X_train_dbth, Y_train)
score = method.score(X_test_dbth, Y_test)
ypred = method.predict(X_test_dbth)
ypred_label = np.array([ACTIVITY_DIC[y] for y in ypred])
te = time.time()
t_total = te-ts

print("Score : %f, time running : %d secondes" %(score, te-ts))
pd.crosstab(Y_test_label, ypred_label, rownames=['True'], colnames=['Pred'])

**Q.** Comparez les performances de ces différentes méthodes avec celles obtenues sur les signaux bruts et celles obtenues sur les données métier. 

# Conclusion 

L'analyse des données brutes (signaux)  ou de leur décomposition sur une base d'ondelettes ne permet pas de retrouver les performances obtenues sur les données métiers. Une autre approche consistera à utiliser l'apprentissage profond sur les  signaux afin d'essayer de retrouver des performances plus proches de celles obtenues avec les données métier. 